In [1]:
import plotly.graph_objects as go
import backtrader as bt
import pandas as pd
import matplotlib.pyplot as plt
import yfinance as yf


In [ ]:

class MyStrategy(bt.Strategy):
    def __init__(self):
        self.sma = bt.indicators.SimpleMovingAverage(period=20)

    def next(self):
        if not self.position and self.data.close > self.sma:
            self.buy()
        elif self.position and self.data.close < self.sma:
            self.sell()

class BuyAndHold(bt.Strategy):
    def __init__(self):
        pass

    def start(self):
        # Buy at the start
        self.buy()

if __name__ == '__main__':
    cerebro = bt.Cerebro()

    # Add your strategy
    cerebro.addstrategy(MyStrategy)
    cerebro.addstrategy(BuyAndHold)

    ticker = "BTC-USD"

    data = yf.download(ticker, start="2024-01-01", end="2024-12-27")

    data.reset_index(inplace=True)
    data.columns = [col[0] if col[1] == '' else col[0] for col in data.columns]


    columns_to_keep = ['Date', 'Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume']
    data = data[columns_to_keep]

    # Check if data was fetched successfully
    if data.empty:
        print("Failed to fetch data for BTC-USD")
    else:

        # Ensure the index is datetime
        data.index = pd.to_datetime(data["Date"])

        # Pass the data to Backtrader
        data_feed = bt.feeds.PandasData(dataname=data)

        # Add data feed to Cerebro
        cerebro.adddata(data_feed)

        # Set initial cash and commission
        cerebro.broker.setcash(200000)
        
        cerebro.broker.set_fund_allocation(0, cash=100000)  # MyStrategy
        cerebro.broker.set_fund_allocation(1, cash=100000)  # BuyAndHold

        cerebro.broker.setcommission(commission=0.001)

        # Add TimeReturn analyzer
        cerebro.addanalyzer(bt.analyzers.TimeReturn, _name="timereturn")
        cerebro.addanalyzer(bt.analyzers.Returns, _name="returns")
        cerebro.addanalyzer(bt.analyzers.DrawDown, _name="drawdown")

        # Run backtest
        results = cerebro.run()
        print(results)

        strategy_results = []
        for strat in results:
            time_return = strat.analyzers.timereturn.get_analysis()
            print(strat.analyzers.returns.get_analysis())
            cumulative_returns = pd.Series(time_return)
            strategy_results.append((strat.__class__.__name__, cumulative_returns))     



In [ ]:
strategy_results

In [ ]:
fig = go.Figure()

for strategy_name, cumulative_returns in strategy_results:
    print(cumulative_returns)
    cumulative_returns = (1 + cumulative_returns).cumprod() - 1  # Calculate cumulative returns
    fig.add_trace(go.Scatter(
        x=cumulative_returns.index,
        y=cumulative_returns.values,
        mode='lines',
        name=strategy_name
    ))

fig.update_layout(
    title="Strategy Comparison: Buy-and-Hold vs. Custom Strategy",
    xaxis_title="Date",
    yaxis_title="Cumulative Returns",
    template="plotly_white",
    height=600,
    width=1000,
)

fig.show()

In [ ]:
import backtrader as bt
import pandas as pd
import yfinance as yf

class MyStrategy(bt.Strategy):
    def __init__(self):
        self.sma = bt.indicators.SimpleMovingAverage(period=20)

    def next(self):
        if not self.position and self.data.close > self.sma:
            self.buy()
        elif self.position and self.data.close < self.sma:
            self.sell()

if __name__ == '__main__':
    cerebro = bt.Cerebro()

    cerebro.addstrategy(MyStrategy)

    data = yf.download("BTC-USD", start="2020-01-01", end="2023-01-01")

    data.reset_index(inplace=True)
    data.columns = [col[0] if col[1] == '' else col[0] for col in data.columns]


    columns_to_keep = ['Date', 'Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume']
    data = data[columns_to_keep]

    data = data.rename(
        columns={
            "Open": "open",
            "High": "high",
            "Low": "low",
            "Close": "close",
            "Adj Close": "adj_close",
            "Volume": "volume",
        }
    )


    data.index = pd.to_datetime(data["Date"])

    data = bt.feeds.PandasData(dataname=data)
    cerebro.adddata(data)

    cerebro.run()

    cerebro.plot(style='candle', barup='green', bardown='red', volume=True, width=14, height=10)


[*********************100%***********************]  1 of 1 completed


In [1]:
import yfinance as yf
import backtrader as bt
import pandas as pd

# Veri indirme
ticker = "JPM"
data = yf.download(ticker, start="2018-01-01", end="2024-11-29")

# NaN ve Inf değerlerini temizleme
data.fillna(method='ffill', inplace=True)
data.replace([float('inf'), float('-inf')], float('nan'), inplace=True)
data.dropna(inplace=True)

# 'Date' sütununu datetime formatına dönüştürme
data.reset_index(inplace=True)
data['Date'] = pd.to_datetime(data['Date'])  # Tarih sütununu datetime formatına dönüştür
data.set_index('Date', inplace=True)  # Backtrader için Date sütununu index yap

# Backtrader'a uygun hale getirme
class PandasData(bt.feeds.PandasData):
    lines = ('volume',)
    params = (('volume', 5),)

data_bt = bt.feeds.PandasData(dataname=data)

# VW-MACD Stratejisi
class VWMACDStrategy(bt.Strategy):
    params = (
        ('short_period', 12),
        ('long_period', 26),
        ('signal_period', 9),
    )

    def __init__(self):
        # VWMA hesaplama
        self.vwma_short = bt.indicators.WeightedMovingAverage(self.data.close, period=self.params.short_period)
        self.vwma_long = bt.indicators.WeightedMovingAverage(self.data.close, period=self.params.long_period)

        # VMACD ve sinyal hattı
        self.vmacd = self.vwma_short - self.vwma_long
        self.signal = bt.indicators.EMA(self.vmacd, period=self.params.signal_period)

    def next(self):
        # Alış sinyali: VMACD > Sinyal Çizgisi
        if self.vmacd[0] > self.signal[0] and not self.position:
            self.buy()

        # Satış sinyali: VMACD < Sinyal Çizgisi
        elif self.vmacd[0] < self.signal[0] and self.position:
            self.sell()

# Cerebro (Backtest motoru)
cerebro = bt.Cerebro()

# Veri ekleme
cerebro.adddata(data_bt)

# Strateji ekleme
cerebro.addstrategy(VWMACDStrategy)

# Başlangıç sermayesi
cerebro.broker.set_cash(10000)

# Komisyon ayarı (opsiyonel)
cerebro.broker.setcommission(commission=0.001)

# Backtest çalıştır
print("Başlangıç Portföy Değeri: %.2f" % cerebro.broker.getvalue())
cerebro.run()
print("Son Portföy Değeri: %.2f" % cerebro.broker.getvalue())

# Grafik oluştur
cerebro.plot()

[*********************100%***********************]  1 of 1 completed
/tmp/ipykernel_14528/3232389769.py:10: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  data.fillna(method='ffill', inplace=True)


Başlangıç Portföy Değeri: 10000.00


/home/arda/anaconda3/envs/algoTrade/lib/python3.12/site-packages/backtrader/cerebro.py:670: SyntaxWarning: invalid escape sequence '\*'
  '''Adds a callback to get messages which would be handled by the
/home/arda/anaconda3/envs/algoTrade/lib/python3.12/site-packages/backtrader/cerebro.py:712: SyntaxWarning: invalid escape sequence '\*'
  '''Adds a callback to get messages which would be handled by the


AttributeError: 'tuple' object has no attribute 'lower'